Capstone Project - The Battle of Neighborhoods 

* Problem section

Background:

Agreeing to Bloomberg News, the London Lodging Advertise is in a groove. It is presently confronting a number of diverse headwinds, counting the prospect of higher charges and a caution from the Bank of Britain that U.K. domestic values seem drop as much as 30 percent within the occasion of a tumultuous exit from the European Union. More particularly, four ignored splits propose that the London advertise may be in more awful shape than numerous realize: covered up cost falls, record-low deals, homebuilder departure and assess climbs tending to abroad buyers of homes in Britain and Ridges.

Business Problem:

In this situation, it is critical to receive machine learning devices in arrange to help homebuyers clientele in London to form shrewd and compelling choices. As a result, the business problem we are right now posturing is: how may we offer back to homebuyers clientele in to buy a appropriate genuine domain in London in this dubious financial and money related scenario? To fathom this trade issue, we are getting to cluster London neighborhoods in arrange to suggest scenes and the current normal cost of genuine bequest where homebuyers can make a genuine bequest venture. We'll prescribe productive scenes concurring to comforts and basic offices encompassing such scenes i.e. rudimentary schools, tall schools, healing centers & basic supply stores.

* Data Section

Information on London properties and the relative cost paid information were extricated from the HM Arrive Registry (http://landregistry.data.gov.uk/). The taking after areas contain the address information included in Cost Paid Information: Postcode; PAON Essential Addressable Protest Title. Regularly the house number or title; SAON Auxiliary Addressable Protest Title. On the off chance that there's a sub-building, for illustration, the building is partitioned into pads, there will be a SAON; Road; Territory; Town/City; Locale; County. To investigate and target suggested areas over distinctive settings concurring to the nearness of comforts and essential facilities, we are going get to information through FourSquare API interface and organize them as a dataframe for visualization. By combining information on London properties and the relative cost paid information from the HM Arrive Registry and information on comforts and essential facilities surrounding such properties from FourSquare API interface, we are going be able to prescribe productive genuine bequest speculations.

* Methodology


This segment will depict the most components of our investigation and predication framework. It comprises four stages:

1. Collect Data
2. Understand Data
3. Data preparation and preprocessing 
4. Modeling

Part 1. Collect Data

Importing Data from HM Land Registry

In [2]:

import os # Operating System
import numpy as np
import pandas as pd
import datetime as dt # Datetime
import json # library to handle JSON files

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

!conda install -c conda-forge folium=0.5.0 --yes
import folium #import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.21.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

In [3]:
#Read the data for examination (Source: http://landregistry.data.gov.uk/)
df_ppd = pd.read_csv("http://prod2.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com/pp-2018.csv")

Part 2. Understand Data

We examined the dataset that we collected into a pandas' information outline and show the primary five columns of it as follows:

In [4]:
df_ppd.head(5)

,{7011B109-CFCA-8ED6-E053-6B04A8C075C1},280000,2018-06-04 00:00,IP4 5ES,S,N,F,3,Unnamed: 8,RANDWELL CLOSE,Unnamed: 10,IPSWICH,IPSWICH.1,SUFFOLK,A,A.1
0,{7011B109-CFCB-8ED6-E053-6B04A8C075C1},280000,2018-05-29 00:00,IP1 4BS,T,N,F,261,NaN,NORWICH ROAD,NaN,IPSWICH,IPSWICH,SUFFOLK,A,A
1,{7011B109-CFCC-8ED6-E053-6B04A8C075C1},170000,2018-04-27 00:00,IP4 4BH,T,N,F,31,NaN,PARADE ROAD,NaN,IPSWICH,IPSWICH,SUFFOLK,A,A
2,{7011B109-CFCD-8ED6-E053-6B04A8C075C1},246000,2018-05-25 00:00,IP1 6NB,S,N,F,42,NaN,ELMCROFT ROAD,NaN,IPSWICH,IPSWICH,SUFFOLK,A,A
3,{7011B109-CFCE-8ED6-E053-6B04A8C075C1},180000,2018-06-08 00:00,IP3 9LZ,T,N,F,48,NaN,WYNTERTON CLOSE,NaN,IPSWICH,IPSWICH,SUFFOLK,A,A
4,{7011B109-CFCF-8ED6-E053-6B04A8C075C1},245000,2018-05-11 00:00,IP1 4BU,T,N,F,235,NaN,NORWICH ROAD,NaN,IPSWICH,IPSWICH,SUFFOLK,A,A


In [5]:
df_ppd.shape

(1027866, 16)

Part 3. Data Preparation and Processing

At this organize, we get ready our dataset for the modeling handle, selecting for the foremost appropriate machine learning calculation for our scope. In like manner, we perform the taking after steps:

a. Rename the column names 

b. Format the date column 

c. Sort information by date of sale 

d. Select information as it were for the city of London 

e. Make a list of road names in London 

f. Calculate the street-wise normal cost of the property 

g. Read the street-wise facilitates into a information outline, dispensing with repeating word London from person names 

h. Join the information to discover the facilitates of areas which fit into client's budget 

i. Plot suggested areas on London outline at the side current showcase costs

In [7]:
# Assign meaningful column names
df_ppd.columns = ['TUID', 'Price', 'Date_Transfer', 'Postcode', 'Prop_Type', 'Old_New', 'Duration', 'PAON', \
                  'SAON', 'Street', 'Locality', 'Town_City', 'District', 'County', 'PPD_Cat_Type', 'Record_Status']

In [8]:

# Format the date column
df_ppd['Date_Transfer'] = df_ppd['Date_Transfer'].apply(pd.to_datetime)

# Delete all obsolete transactions which were done before 2016
df_ppd.drop(df_ppd[df_ppd.Date_Transfer.dt.year < 2016].index, inplace=True)

# Sort by Date of Sale
df_ppd.sort_values(by=['Date_Transfer'],ascending=[False],inplace=True)

In [9]:
df_ppd_london = df_ppd.query("Town_City == 'LONDON'")

# Make a list of street names in LONDON
streets = df_ppd_london['Street'].unique().tolist()

In [10]:

df_grp_price = df_ppd_london.groupby(['Street'])['Price'].mean().reset_index()

# Give meaningful names to the columns
df_grp_price.columns = ['Street', 'Avg_Price']

In [11]:
#Input your Budget's Upper Limit and Lower Limit - Find the locations df_grp_price which fits your budget
df_affordable = df_grp_price.query("(Avg_Price >= 2200000) & (Avg_Price <= 2500000)")

In [12]:

# Display the dataframe
df_affordable

,Street,Avg_Price
196,ALBION SQUARE,2.450000e+06
390,ANHALT ROAD,2.435000e+06
405,ANSDELL TERRACE,2.250000e+06
422,APPLEGARTH ROAD,2.400000e+06
855,BARONSMEAD ROAD,2.375000e+06
981,BEAUCLERC ROAD,2.480000e+06
1102,BELVEDERE DRIVE,2.340000e+06
1215,BICKENHALL STREET,2.208500e+06
1253,BIRCHLANDS AVENUE,2.217000e+06
1553,BRAMPTON GROVE,2.456875e+06


In [13]:

import pandas as pd
import numpy as np
import datetime as DT
import hmac
from geopy.geocoders import Nominatim
from geopy.distance import vincenty
# import k-means from clustering stage
from sklearn.cluster import KMeans

In [14]:
for index, item in df_affordable.iterrows():
    print(f"index: {index}")
    print(f"item: {item}")
    print(f"item.Street only: {item.Street}")

index: 196
item: Street       ALBION SQUARE
Avg_Price         2.45e+06
Name: 196, dtype: object
item.Street only: ALBION SQUARE
index: 390
item: Street       ANHALT ROAD
Avg_Price      2.435e+06
Name: 390, dtype: object
item.Street only: ANHALT ROAD
index: 405
item: Street       ANSDELL TERRACE
Avg_Price           2.25e+06
Name: 405, dtype: object
item.Street only: ANSDELL TERRACE
index: 422
item: Street       APPLEGARTH ROAD
Avg_Price            2.4e+06
Name: 422, dtype: object
item.Street only: APPLEGARTH ROAD
index: 855
item: Street       BARONSMEAD ROAD
Avg_Price          2.375e+06
Name: 855, dtype: object
item.Street only: BARONSMEAD ROAD
index: 981
item: Street       BEAUCLERC ROAD
Avg_Price          2.48e+06
Name: 981, dtype: object
item.Street only: BEAUCLERC ROAD
index: 1102
item: Street       BELVEDERE DRIVE
Avg_Price           2.34e+06
Name: 1102, dtype: object
item.Street only: BELVEDERE DRIVE
index: 1215
item: Street       BICKENHALL STREET
Avg_Price           2.2085e+06
N

In [15]:
geolocator = Nominatim()

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: DeprecationWarning: Using Nominatim with the default "geopy/1.21.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  if __name__ == '__main__':


In [16]:

df_affordable['city_coord'] = df_affordable['Street'].apply(geolocator.geocode).apply(lambda x: (x.latitude, x.longitude))

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [17]:
df_affordable

,Street,Avg_Price,city_coord
196,ALBION SQUARE,2.450000e+06,"(-41.27375755, 173.28939323910353)"
390,ANHALT ROAD,2.435000e+06,"(51.4803164, -0.1668011)"
405,ANSDELL TERRACE,2.250000e+06,"(51.4998899, -0.1891027)"
422,APPLEGARTH ROAD,2.400000e+06,"(53.7486539, -0.3266704)"
855,BARONSMEAD ROAD,2.375000e+06,"(51.4773147, -0.239457)"
981,BEAUCLERC ROAD,2.480000e+06,"(30.2114523, -81.6179807)"
1102,BELVEDERE DRIVE,2.340000e+06,"(52.4142089, 1.7244152)"
1215,BICKENHALL STREET,2.208500e+06,"(51.5212014, -0.1589082)"
1253,BIRCHLANDS AVENUE,2.217000e+06,"(51.4483941, -0.1604676)"
1553,BRAMPTON GROVE,2.456875e+06,"(51.5899607, -0.3185249)"


In [18]:
df_affordable[['Latitude', 'Longitude']] = df_affordable['city_coord'].apply(pd.Series)

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/frame.py:3391: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [19]:
df_affordable

,Street,Avg_Price,city_coord,Latitude,Longitude
196,ALBION SQUARE,2.450000e+06,"(-41.27375755, 173.28939323910353)",-41.273758,173.289393
390,ANHALT ROAD,2.435000e+06,"(51.4803164, -0.1668011)",51.480316,-0.166801
405,ANSDELL TERRACE,2.250000e+06,"(51.4998899, -0.1891027)",51.499890,-0.189103
422,APPLEGARTH ROAD,2.400000e+06,"(53.7486539, -0.3266704)",53.748654,-0.326670
855,BARONSMEAD ROAD,2.375000e+06,"(51.4773147, -0.239457)",51.477315,-0.239457
981,BEAUCLERC ROAD,2.480000e+06,"(30.2114523, -81.6179807)",30.211452,-81.617981
1102,BELVEDERE DRIVE,2.340000e+06,"(52.4142089, 1.7244152)",52.414209,1.724415
1215,BICKENHALL STREET,2.208500e+06,"(51.5212014, -0.1589082)",51.521201,-0.158908
1253,BIRCHLANDS AVENUE,2.217000e+06,"(51.4483941, -0.1604676)",51.448394,-0.160468
1553,BRAMPTON GROVE,2.456875e+06,"(51.5899607, -0.3185249)",51.589961,-0.318525


In [20]:
df = df_affordable.drop(columns=['city_coord'])

In [21]:

df

,Street,Avg_Price,Latitude,Longitude
196,ALBION SQUARE,2.450000e+06,-41.273758,173.289393
390,ANHALT ROAD,2.435000e+06,51.480316,-0.166801
405,ANSDELL TERRACE,2.250000e+06,51.499890,-0.189103
422,APPLEGARTH ROAD,2.400000e+06,53.748654,-0.326670
855,BARONSMEAD ROAD,2.375000e+06,51.477315,-0.239457
981,BEAUCLERC ROAD,2.480000e+06,30.211452,-81.617981
1102,BELVEDERE DRIVE,2.340000e+06,52.414209,1.724415
1215,BICKENHALL STREET,2.208500e+06,51.521201,-0.158908
1253,BIRCHLANDS AVENUE,2.217000e+06,51.448394,-0.160468
1553,BRAMPTON GROVE,2.456875e+06,51.589961,-0.318525


In [22]:

address = 'London, UK'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of London City are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:4: DeprecationWarning: Using Nominatim with the default "geopy/1.21.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.


The geograpical coordinate of London City are 51.5073219, -0.1276474.


In [23]:
# create map of London using latitude and longitude values
map_london = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, price, street in zip(df['Latitude'], df['Longitude'], df['Avg_Price'], df['Street']):
    label = '{}, {}'.format(street, price)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london)  
    
map_london

In [24]:
#Define Foursquare Credentials and Version

CLIENT_ID = 'KI3TR0QO4JOKMFELOMF3WSOOI3HFNBF5YLW354MYWBKDHEX3' # Foursquare ID
CLIENT_SECRET = 'QF4ZBLJRBV4BQX52DVWUPEHJ14A2UJABPCZARZQZYTKIISUD' # Foursquare Secret
VERSION = '20181206' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: KI3TR0QO4JOKMFELOMF3WSOOI3HFNBF5YLW354MYWBKDHEX3
CLIENT_SECRET:QF4ZBLJRBV4BQX52DVWUPEHJ14A2UJABPCZARZQZYTKIISUD


We can presently continue to the Modeling stage. We'll analyze neighborhoods to suggest genuine domains where domestic buyers can make a genuine bequest venture. We are going at that point prescribe productive scenes concurring to civilities and basic offices encompassing such settings i.e. basic schools, tall schools, healing centers & basic supply stores.

Part 4. Modeling

After investigating the dataset and picking up experiences into it, we are prepared to utilize the clustering technique to analyze genuine bequests. We are going use the k-means clustering strategy because it is quick and effective in terms of computational taken a toll, is exceedingly adaptable to account for transformations in genuine bequest showcase in London and is exact.

In [25]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Street', 
                  'Street Latitude', 
                  'Street Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [26]:

# Run the above function on each location and create a new dataframe called location_venues and display it.
location_venues = getNearbyVenues(names=df['Street'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

ALBION SQUARE
ANHALT ROAD
ANSDELL TERRACE
APPLEGARTH ROAD
BARONSMEAD ROAD
BEAUCLERC ROAD
BELVEDERE DRIVE
BICKENHALL STREET
BIRCHLANDS AVENUE
BRAMPTON GROVE
BRIARDALE GARDENS
BROOKWAY
BURBAGE ROAD
BURY WALK
CALLCOTT STREET
CAMPDEN HILL ROAD
CAMPION ROAD
CANNING PLACE
CARLISLE ROAD
CARLTON GARDENS
CARLYLE COURT
CHALCOT SQUARE
CHARLES LANE
CHELSEA CRESCENT
CHESTER CLOSE NORTH
CHEYNE COURT
CHEYNE ROW
CHISWICK MALL
CITY ROAD
CLARENDON STREET
CLONCURRY STREET
COLBECK MEWS
COLLEGE CRESCENT
CORNWALL TERRACE MEWS
COURT LANE GARDENS
CRESCENT GROVE
DALEBURY ROAD
DEWHURST ROAD
DORIA ROAD
DOWNSHIRE HILL
DUCHESS WALK
ECCLESTON SQUARE MEWS
EGBERT STREET
EGERTON PLACE
ELM PARK ROAD
FLORAL STREET
FRANK DIXON WAY
FULTON MEWS
GERARD ROAD
GERRARD ROAD
GIRDLERS ROAD
GLOUCESTER CRESCENT
GORDON PLACE
GRAFTON SQUARE
GRAHAM TERRACE
HARMAN DRIVE
HARRIS STREET
HAVANNAH STREET
HAZLEWELL ROAD
HEREFORD MEWS
HERONDALE AVENUE
HIGHGATE HIGH STREET
HIGHWOOD HILL
HILLGATE PLACE
HOLLYCROFT AVENUE
HOLLYWOOD MEWS
HONEYWELL

In [27]:
location_venues

,Street,Street Latitude,Street Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,ALBION SQUARE,-41.273758,173.289393,The Free House,-41.273340,173.287364,Bar
1,ALBION SQUARE,-41.273758,173.289393,The Indian Cafe,-41.273308,173.286530,Indian Restaurant
2,ALBION SQUARE,-41.273758,173.289393,Queen's Gardens,-41.273671,173.291383,Park
3,ALBION SQUARE,-41.273758,173.289393,Deville Cafe,-41.271941,173.285535,Beer Garden
4,ALBION SQUARE,-41.273758,173.289393,Urban,-41.274355,173.286317,New American Restaurant
5,ALBION SQUARE,-41.273758,173.289393,The Bridge Street Collective,-41.272520,173.285517,Café
6,ALBION SQUARE,-41.273758,173.289393,Burger Culture,-41.274750,173.284030,Burger Joint
7,ALBION SQUARE,-41.273758,173.289393,Fish Stop,-41.276010,173.289592,Fish & Chips Shop
8,ALBION SQUARE,-41.273758,173.289393,The Vic Mac's Brew Bar,-41.274757,173.283914,Pub
9,ALBION SQUARE,-41.273758,173.289393,Sprig & Fern,-41.274508,173.286527,Brewery


In [28]:
location_venues.groupby('Street').count()

,Street Latitude,Street Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Street,,,,,,
ALBION SQUARE,30,30,30,30,30,30
ANHALT ROAD,15,15,15,15,15,15
ANSDELL TERRACE,50,50,50,50,50,50
APPLEGARTH ROAD,4,4,4,4,4,4
BARONSMEAD ROAD,13,13,13,13,13,13
BEAUCLERC ROAD,5,5,5,5,5,5
BELVEDERE DRIVE,5,5,5,5,5,5
BICKENHALL STREET,90,90,90,90,90,90
BIRCHLANDS AVENUE,10,10,10,10,10,10


In [29]:
# get the List of Unique Categories
print('There are {} uniques categories.'.format(len(location_venues['Venue Category'].unique())))

There are 352 uniques categories.


In [30]:
location_venues.shape

(5861, 7)

In [31]:
# one hot encoding
venues_onehot = pd.get_dummies(location_venues[['Venue Category']], prefix="", prefix_sep="")

# add street column back to dataframe
venues_onehot['Street'] = location_venues['Street'] 

# move street column to the first column
fixed_columns = [venues_onehot.columns[-1]] + list(venues_onehot.columns[:-1])

#fixed_columns
venues_onehot = venues_onehot[fixed_columns]

venues_onehot.head()

,Street,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,...,Video Store,Vietnamese Restaurant,Warehouse Store,Waterfront,Windmill,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo
0,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
london_grouped = venues_onehot.groupby('Street').mean().reset_index()
london_grouped

,Street,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,...,Video Store,Vietnamese Restaurant,Warehouse Store,Waterfront,Windmill,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo
0,ALBION SQUARE,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.033333,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
1,ANHALT ROAD,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
2,ANSDELL TERRACE,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.020000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
3,APPLEGARTH ROAD,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
4,BARONSMEAD ROAD,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
5,BEAUCLERC ROAD,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
6,BELVEDERE DRIVE,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
7,BICKENHALL STREET,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.011111,0.000000,0.000000,0.011111,0.0
8,BIRCHLANDS AVENUE,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
9,BRAMPTON GROVE,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0


In [33]:
london_grouped.shape

(155, 353)

In [34]:
# What are the top 5 venues/facilities nearby profitable real estate investments?#

num_top_venues = 5

for hood in london_grouped['Street']:
    print("----"+hood+"----")
    temp = london_grouped[london_grouped['Street'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----ALBION SQUARE----
         venue  freq
0         Café  0.20
1  Coffee Shop  0.10
2          Bar  0.07
3          Pub  0.07
4   Restaurant  0.07


----ANHALT ROAD----
                 venue  freq
0                  Pub  0.27
1        Grocery Store  0.13
2  Japanese Restaurant  0.07
3         Cocktail Bar  0.07
4               Garden  0.07


----ANSDELL TERRACE----
               venue  freq
0                Pub  0.06
1         Restaurant  0.06
2     Clothing Store  0.06
3              Hotel  0.06
4  French Restaurant  0.04


----APPLEGARTH ROAD----
               venue  freq
0                Bar  0.50
1          Nightclub  0.25
2             Casino  0.25
3  Outdoor Sculpture  0.00
4               Park  0.00


----BARONSMEAD ROAD----
               venue  freq
0    Nature Preserve  0.08
1     Breakfast Spot  0.08
2     Farmers Market  0.08
3        Coffee Shop  0.08
4  Food & Drink Shop  0.08


----BEAUCLERC ROAD----
             venue  freq
0  Harbor / Marina   0.2
1        Speakeas

In [35]:
# Define a function to return the most common venues/facilities nearby real estate investments#

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [36]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Street']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

In [37]:
# create a new dataframe
venues_sorted = pd.DataFrame(columns=columns)
venues_sorted['Street'] = london_grouped['Street']

for ind in np.arange(london_grouped.shape[0]):
    venues_sorted.iloc[ind, 1:] = return_most_common_venues(london_grouped.iloc[ind, :], num_top_venues)

In [38]:

venues_sorted.head()

,Street,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,ALBION SQUARE,Café,Coffee Shop,Pub,Bar,Indian Restaurant,Restaurant,French Restaurant,Supermarket,Beer Garden,Art Gallery
1,ANHALT ROAD,Pub,Grocery Store,Gym / Fitness Center,Plaza,Pizza Place,Diner,Cocktail Bar,English Restaurant,Garden,French Restaurant
2,ANSDELL TERRACE,Pub,Clothing Store,Restaurant,Hotel,Indian Restaurant,English Restaurant,Bakery,French Restaurant,Italian Restaurant,Café
3,APPLEGARTH ROAD,Bar,Casino,Nightclub,Zoo,Film Studio,Exhibit,Factory,Falafel Restaurant,Farm,Farmers Market
4,BARONSMEAD ROAD,Indie Movie Theater,Restaurant,Food & Drink Shop,Breakfast Spot,Nature Preserve,Farmers Market,Thai Restaurant,Park,Coffee Shop,Movie Theater


In [39]:
venues_sorted.shape

(155, 11)

In [40]:

london_grouped.shape

(155, 353)

In [41]:
london_grouped=df

In [42]:
#Distribute in 5 Clusters

# set number of clusters
kclusters = 5

london_grouped_clustering = london_grouped.drop('Street', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(london_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:50]

array([1, 3, 0, 3, 2, 1, 2, 0, 0, 1, 3, 3, 3, 1, 2, 2, 1, 3, 0, 1, 4, 4,
       3, 1, 1, 0, 3, 4, 1, 0, 3, 2, 3, 2, 2, 4, 3, 3, 2, 0, 1, 2, 4, 0,
       4, 0, 0, 4, 0, 0], dtype=int32)

In [43]:
#Dataframe to include Clusters

london_grouped_clustering=df
london_grouped_clustering.head()

,Street,Avg_Price,Latitude,Longitude
196,ALBION SQUARE,2450000.0,-41.273758,173.289393
390,ANHALT ROAD,2435000.0,51.480316,-0.166801
405,ANSDELL TERRACE,2250000.0,51.499890,-0.189103
422,APPLEGARTH ROAD,2400000.0,53.748654,-0.326670
855,BARONSMEAD ROAD,2375000.0,51.477315,-0.239457


In [44]:
london_grouped_clustering.shape

(162, 4)

In [45]:
df.shape

(162, 4)

In [46]:
london_grouped_clustering.dtypes

Street        object
Avg_Price    float64
Latitude     float64
Longitude    float64
dtype: object

In [47]:

df.dtypes

Street        object
Avg_Price    float64
Latitude     float64
Longitude    float64
dtype: object

In [48]:
# add clustering labels
london_grouped_clustering['Cluster Labels'] = kmeans.labels_

# merge london_grouped with london_data to add latitude/longitude for each neighborhood
london_grouped_clustering = london_grouped_clustering.join(venues_sorted.set_index('Street'), on='Street')

london_grouped_clustering.head(30) # check the last columns!

,Street,Avg_Price,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
196,ALBION SQUARE,2.450000e+06,-41.273758,173.289393,1,Café,Coffee Shop,Pub,Bar,Indian Restaurant,Restaurant,French Restaurant,Supermarket,Beer Garden,Art Gallery
390,ANHALT ROAD,2.435000e+06,51.480316,-0.166801,3,Pub,Grocery Store,Gym / Fitness Center,Plaza,Pizza Place,Diner,Cocktail Bar,English Restaurant,Garden,French Restaurant
405,ANSDELL TERRACE,2.250000e+06,51.499890,-0.189103,0,Pub,Clothing Store,Restaurant,Hotel,Indian Restaurant,English Restaurant,Bakery,French Restaurant,Italian Restaurant,Café
422,APPLEGARTH ROAD,2.400000e+06,53.748654,-0.326670,3,Bar,Casino,Nightclub,Zoo,Film Studio,Exhibit,Factory,Falafel Restaurant,Farm,Farmers Market
855,BARONSMEAD ROAD,2.375000e+06,51.477315,-0.239457,2,Indie Movie Theater,Restaurant,Food & Drink Shop,Breakfast Spot,Nature Preserve,Farmers Market,Thai Restaurant,Park,Coffee Shop,Movie Theater
981,BEAUCLERC ROAD,2.480000e+06,30.211452,-81.617981,1,Spa,Speakeasy,Harbor / Marina,Pizza Place,Automotive Shop,Zoo,Fast Food Restaurant,Event Space,Exhibit,Factory
1102,BELVEDERE DRIVE,2.340000e+06,52.414209,1.724415,2,Pub,Seafood Restaurant,Fish & Chips Shop,Campground,Vacation Rental,Farmers Market,Ethiopian Restaurant,Event Space,Exhibit,Factory
1215,BICKENHALL STREET,2.208500e+06,51.521201,-0.158908,0,Coffee Shop,Café,Hotel,Indian Restaurant,Chinese Restaurant,Restaurant,Pizza Place,Gastropub,Bakery,Thai Restaurant
1253,BIRCHLANDS AVENUE,2.217000e+06,51.448394,-0.160468,0,Pub,Coffee Shop,Brewery,Breakfast Spot,Chinese Restaurant,Bakery,Lake,Train Station,French Restaurant,Dessert Shop
1553,BRAMPTON GROVE,2.456875e+06,51.589961,-0.318525,1,Home Service,Zoo,English Restaurant,Event Space,Exhibit,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant


In [49]:
# Create Map

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(london_grouped_clustering['Latitude'], london_grouped_clustering['Longitude'], london_grouped_clustering['Street'], london_grouped_clustering['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [50]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 0, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
405,2250000.0,Pub,Clothing Store,Restaurant,Hotel,Indian Restaurant,English Restaurant,Bakery,French Restaurant,Italian Restaurant,Café
1215,2208500.0,Coffee Shop,Café,Hotel,Indian Restaurant,Chinese Restaurant,Restaurant,Pizza Place,Gastropub,Bakery,Thai Restaurant
1253,2217000.0,Pub,Coffee Shop,Brewery,Breakfast Spot,Chinese Restaurant,Bakery,Lake,Train Station,French Restaurant,Dessert Shop
2225,2200000.0,Liquor Store,Indian Restaurant,Convenience Store,Pizza Place,Bakery,Café,Fish & Chips Shop,Sandwich Place,Pharmacy,Farmers Market
2637,2250000.0,Construction & Landscaping,Gastropub,Zoo,Fish & Chips Shop,Exhibit,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant


In [51]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 1, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
196,2450000.0,Café,Coffee Shop,Pub,Bar,Indian Restaurant,Restaurant,French Restaurant,Supermarket,Beer Garden,Art Gallery
981,2480000.0,Spa,Speakeasy,Harbor / Marina,Pizza Place,Automotive Shop,Zoo,Fast Food Restaurant,Event Space,Exhibit,Factory
1553,2456875.0,Home Service,Zoo,English Restaurant,Event Space,Exhibit,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
1980,2492500.0,Supermarket,English Restaurant,Park,Fast Food Restaurant,Café,Hardware Store,Dry Cleaner,Gym,Rental Car Location,American Restaurant
2136,2461000.0,Soccer Field,Sports Club,Windmill,Film Studio,Event Space,Exhibit,Factory,Falafel Restaurant,Farm,Farmers Market


In [52]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 2, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
855,2375000.0,Indie Movie Theater,Restaurant,Food & Drink Shop,Breakfast Spot,Nature Preserve,Farmers Market,Thai Restaurant,Park,Coffee Shop,Movie Theater
1102,2340000.0,Pub,Seafood Restaurant,Fish & Chips Shop,Campground,Vacation Rental,Farmers Market,Ethiopian Restaurant,Event Space,Exhibit,Factory
2068,2375000.0,Pub,Park,Grocery Store,Café,Indian Restaurant,Pizza Place,Yoga Studio,Hotel,Portuguese Restaurant,English Restaurant
2129,2379652.7,Café,Bakery,Italian Restaurant,Clothing Store,Modern European Restaurant,Gym / Fitness Center,Juice Bar,Grocery Store,English Restaurant,Restaurant
2943,2367500.0,Hotel,Pub,Garden,Café,Italian Restaurant,Coffee Shop,Chinese Restaurant,Bar,Mediterranean Restaurant,Burger Joint


In [53]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 3, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
390,2435000.0,Pub,Grocery Store,Gym / Fitness Center,Plaza,Pizza Place,Diner,Cocktail Bar,English Restaurant,Garden,French Restaurant
422,2400000.0,Bar,Casino,Nightclub,Zoo,Film Studio,Exhibit,Factory,Falafel Restaurant,Farm,Farmers Market
1632,2397132.0,Sporting Goods Shop,Health & Beauty Service,Grocery Store,Coffee Shop,Gym / Fitness Center,Food,Ethiopian Restaurant,Food Stand,Event Space,Exhibit
1797,2400000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1914,2445000.0,Furniture / Home Store,Bakery,Grocery Store,Food & Drink Shop,Pub,Italian Restaurant,Pizza Place,Gastropub,Art Gallery,Stadium


In [54]:

london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 4, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2242,2.300000e+06,Farm,Soup Place,Zoo,Film Studio,Event Space,Exhibit,Factory,Falafel Restaurant,Farmers Market,Fast Food Restaurant
2405,2.286679e+06,Café,Italian Restaurant,Coffee Shop,Bar,Pub,French Restaurant,Park,Beer Store,Belgian Restaurant,Fish Market
2685,2.287500e+06,Pub,Brewery,Italian Restaurant,Reservoir,Art Museum,Gift Shop,Filipino Restaurant,Event Space,Exhibit,Factory
3376,2.298000e+06,Hotel,Zoo,Film Studio,Event Space,Exhibit,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
4284,2.265000e+06,Pub,Convenience Store,Shopping Mall,Filipino Restaurant,Event Space,Exhibit,Factory,Falafel Restaurant,Farm,Farmers Market


Results 

To begin with, we may look at them agreeing to neighborhoods/London regions. It is curiously to note that, in spite of the fact that West London (Notting Slope, Kensington, Chelsea, Marylebone) and North-West London (Hampsted) may well be considered exceedingly productive scenes to buy a genuine domain agreeing to civilities and fundamental offices encompassing such settings i.e. basic schools, tall schools, healing centers & basic supply stores, South-West London (Wandsworth, Balham) and North-West London (Isliington) are emerging as another future tip top scenes with a wide run of civilities and offices. In like manner, one might target under-priced genuine estates in these zones of London in arrange to create a trade affair. Second, we may analyze our comes about concurring to the five clusters we have delivered. Indeed in spite of the fact that, all clusters seem laud an optimal extend of offices and civilities.

Conclusion

To whole up, agreeing to Bloomberg News, the London Lodging Showcase is in a trench. It is presently confronting a number of diverse headwinds, counting the prospect of higher charges and a caution from the Bank of Britain that U.K. domestic values might drop as much as 30 percent within the occasion of a scattered exit from the European Union. In this situation, it is pressing to receive machine learning tools in arrange to help homebuyers clientele in London to create shrewd and viable choices.